### CNN network를 활용하여 이미지간 유사도
: pixel값을 column으로 하여 유사도를 구한다.
1. training을 시킨 model의 _dense를 추출하여 유사도를 구한다.
2. CNN을 통과시켜 feature의 차원이 축소된 이미지 특성값으로 유사도를 구한다.

In [4]:
import tensorflow as tf
from PIL import Image as pil
from scipy.misc import imread, imresize
import os
from scipy import ndimage
from six.moves import urllib

cwd = os.getcwd()
cwd

'C:\\Users\\Juno\\Desktop\\juno_project\\Webtoon-Recommender_System'

In [89]:
# 웹툰 labeling을 위한 dictionary를 만든다.

# Load info
info = pd.read_csv("data_Juno/webtoon_naver_info_by_genres.csv")

# make dict
dic = {}
for i, id in enumerate(info.unique_id):
    dic[id] = i

path = cwd + "./0_img/main_image_naver/color/"
print(path)
flist = os.listdir(path)
n_class = len(dic)
imgcnt = 0
for f in flist:
    fullpath = os.path.join(path,f)
    currimg = np.asarray(pil.open(fullpath).convert("RGB"))
    img = currimg / 255.
#     print(img.shape)
#     print(imresize(img, (64,64)).shape)
#     print(img)
    img_r = imresize(img, (32,32))
    imgvec = np.reshape(img_r, (1,-1))
    
    # make labels
    id_check = int(f.split(".")[0])
    label = dic[id_check]
    curr_label = np.eye(n_class, n_class)[label:label+1, :]
    if imgcnt == 0:
        totalimg = imgvec
        totallabel = curr_label
    else:
        totalimg = np.concatenate((totalimg, imgvec), axis=0)
        totallabel = np.concatenate((totallabel, curr_label), axis=0)
    print("now: %s finished!!:" % imgcnt, end='\r')
    imgcnt += 1
print ("Total %d images loaded." % (imgcnt))

C:\Users\Juno\Desktop\juno_project\Webtoon-Recommender_System./0_img/main_image_naver/color/


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Total 745 images loaded.


In [5]:
n_input = 3072
n_channel = 32 # convolution filter의 갯수
n_classes = 745
n_imgch = 3
x = tf.placeholder(dtype='float', shape=[None, n_input])
y = tf.placeholder(dtype='float', shape=[None, n_classes])

stddev = 0.1
# convolution filters
weights = {
    'c1' : tf.Variable(initial_value=tf.random_normal(shape=[3,3,n_imgch,n_channel], stddev=stddev)),
    'c2' : tf.Variable(initial_value=tf.random_normal(shape=[3,3,n_channel,32], stddev=stddev)),
    'c3' : tf.Variable(initial_value=tf.random_normal(shape=[3,3,32,32], stddev=stddev)),
    'd1' : tf.Variable(initial_value=tf.random_normal(shape=[4*4*32,n_classes], stddev=stddev)),
}
biases = {
    'c1' : tf.Variable(initial_value=tf.random_normal(shape=[n_channel], stddev=stddev)),
    'c2' : tf.Variable(initial_value=tf.random_normal(shape=[32], stddev=stddev)),
    'c3' : tf.Variable(initial_value=tf.random_normal(shape=[32], stddev=stddev)),
    'd1' : tf.Variable(initial_value=tf.random_normal(shape=[n_classes], stddev=stddev)),
}
print("Model is defined!!")

Model is defined!!


In [6]:
def CNN(_x, _w, _b):
    # Reshape
    _x_r = tf.reshape(_x, shape=[-1,32,32,3])
    # Convolution
    _conv1 = tf.nn.conv2d(_x_r, _w['c1'], strides=[1,1,1,1], padding='SAME') # zero-padding
    # add bias
    _conv2 = tf.nn.bias_add(_conv1, _b['c1'])
    # Passing the ReLU
    _conv3 = tf.nn.relu(_conv2)
    # max pooling
    # non-overlapping
    _pool = tf.nn.max_pool(_conv3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # 2nd Convolutional Layer
    _second = tf.nn.conv2d(_pool, _w['c2'], strides=[1,1,1,1], padding='SAME')
    _second = tf.nn.bias_add(_second, _b['c2'])
    _second = tf.nn.relu(_second)
    _second = tf.nn.max_pool(_second, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # 3rd Convolutional Layer
    _third = tf.nn.conv2d(_second, _w['c3'], strides=[1,1,1,1], padding='SAME')
    _third = tf.nn.bias_add(_third, _b['c3'])
    _third = tf.nn.relu(_third)
    _third = tf.nn.max_pool(_third, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # fully connected layer, 한 줄로 쫙 편다.
    _dense = tf.reshape(_third, shape=[-1,  _w['d1'].get_shape().as_list()[0]])
    _logit = tf.add(tf.matmul(_dense, _w['d1']),_b['d1'])
    
    # Return to dict type
    _out = {
        'x_r':_x_r, 'conv1':_conv1, 'conv2' : _conv2, 'conv3': _conv3,
        'pool': _pool, 'dense': _dense, 'logit':_logit
    }
    return _out

In [13]:
cnnout = CNN(x, weights, biases)
features = cnnout['dense']

**1. training을 시킨 model의 _dense를 추출하여 유사도를 구한다.**

In [15]:
savedir = "./3_NeuralNet_image/main_image/"
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
best_epoch = 9
restorename = savedir + "net-" + str(best_epoch) + ".ckpt"
print ("LOADING [%s]" % (restorename))
saver.restore(sess, restorename)
result = sess.run(features, feed_dict={x:totalimg})


LOADING [./3_NeuralNet_image/main_image/net-9.ckpt]
INFO:tensorflow:Restoring parameters from ./3_NeuralNet_image/main_image/net-9.ckpt


In [57]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity, euclidean_distances

In [39]:
result[0].reshape((1,-1)).shape, result.shape

((1, 512), (745, 512))

In [60]:
cosine_similar = linear_kernel(result[0].reshape((1,-1)), result).flatten()
# 위 코드는 아래와 같은 역할을 한다
# cosine_similar = (srch_vector*X.T).toarray().flatten()

# 유사한 rawdata index
sim_rank_idx = cosine_similar.argsort()[::-1]
sim_rank_idx[:5]

array([498,  74,  73, 226,  51], dtype=int64)

In [62]:
cosine_similar = euclidean_distances(result[1].reshape((1,-1)), result).flatten()
sim_rank_idx = cosine_similar.argsort()
sim_rank_idx[:5]

array([  1, 622, 552, 676, 593], dtype=int64)

---
**2. CNN을 통과시켜 feature의 차원이 축소된 이미지 특성값으로 유사도를 구한다.**

In [91]:
total_image = np.array(totalimg, dtype='float32') / 255.

In [92]:
total_image.shape

(745, 3072)

In [96]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [97]:
cnnout = CNN(x, weights, biases)
features = cnnout['dense']
result = sess.run(features, feed_dict={x:total_image})

In [99]:
result.shape

(745, 512)

In [106]:
euclidean_dist = euclidean_distances(result[1].reshape((1,-1)), result).flatten()
sim_rank_idx = euclidean_dist.argsort()
sim_rank_idx[:10]

array([  1,   0, 325, 552, 651, 250, 167, 516, 160, 432], dtype=int64)

In [101]:
dic2 = {}
for k, v in dic.items():
    dic2[v] = k

In [107]:
dic2[250]

670143

**둘 다 크게 좋은 결과를 도출해내지 못하였다.**

---
**3.tensorflow_hub을 이용하여 이미지 특징 도출 후 similarity 계산**

In [5]:
import tensorflow as tf
import os
cwd = os.getcwd()

In [2]:
import tensorflow_hub as hub
module = hub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_100_96/feature_vector/2")

INFO:tensorflow:Using C:\Users\Juno\AppData\Local\Temp\tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/beta:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/Conv/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/gamma:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/Conv/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/Conv/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tf

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_1/expand/weights:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_1/expand/weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_1/project/BatchNorm/beta:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_1/project/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_1/project/BatchNorm/gamma:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_1/project/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_1/project/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Loc

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_11/expand/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_11/expand/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_11/expand/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_11/expand/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_11/expand/weights:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_11/expand/weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_11/project/BatchNorm/beta:0 from checkpoint b

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_13/expand/BatchNorm/beta:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_13/expand/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_13/expand/BatchNorm/gamma:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_13/expand/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_13/expand/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_13/expand/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_13/expand/BatchNorm/moving_variance:0 from checkpoi

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_15/depthwise/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_15/depthwise/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_15/depthwise/depthwise_weights:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_15/depthwise/depthwise_weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_15/expand/BatchNorm/beta:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_15/expand/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_15/expand/BatchNorm/gamma:0

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_2/depthwise/BatchNorm/gamma:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_2/depthwise/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_2/depthwise/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_2/depthwise/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_2/depthwise/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_2/depthwise/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_2/depthwise/depth

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_3/project/weights:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_3/project/weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_4/depthwise/BatchNorm/beta:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_4/depthwise/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_4/depthwise/BatchNorm/gamma:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_4/depthwise/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_4/depthwise/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\Juno\\

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_5/project/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_5/project/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_5/project/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_5/project/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_5/project/weights:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_5/project/weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_6/depthwise/BatchNorm/beta:0 from checkpoint 

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_7/project/BatchNorm/beta:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_7/project/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_7/project/BatchNorm/gamma:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_7/project/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_7/project/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_7/project/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_7/project/BatchNorm/moving_variance:0 from checkpoi

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_9/expand/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_9/expand/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_9/expand/weights:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_9/expand/weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_9/project/BatchNorm/beta:0 from checkpoint b'C:\\Users\\Juno\\AppData\\Local\\Temp\\tfhub_modules\\3f2c9799c0f3aa9c272fc55e33d45daf10a9b3a0\\variables\\variables' with MobilenetV2/expanded_conv_9/project/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_9/project/BatchNorm/gamma:0 from checkpoint b'C:\\Users\\Juno\\

In [3]:
import os
import PIL.Image as pil 
from scipy.misc import imread, imresize 

In [7]:
# Load info
info = pd.read_csv("data_Juno/webtoon_naver_info_by_genres.csv")

# make dict
dic = {}
for i, id in enumerate(info.unique_id):
    dic[id] = i

# Get main image vectors
path = cwd + "./0_Naverimages/main_image_naver/color/"
print(path)
flist = os.listdir(path)
n_class = len(dic)
imgcnt = 0
for f in flist:
    fullpath = os.path.join(path,f)
    currimg = np.asarray(pil.open(fullpath).convert("RGB"))
    img = currimg / 255.
#     print(img.shape)
#     print(imresize(img, (64,64)).shape)
#     print(img)
    img_r = imresize(img, (96,96)) # mobileNet 고정 input size : (96,96)
    imgvec = np.reshape(img_r, (1,-1))
    
    # make labels
    id_check = int(f.split(".")[0])
    label = dic[id_check]
    curr_label = np.eye(n_class, n_class)[label:label+1, :]
    if imgcnt == 0:
        totalimg = imgvec
        totallabel = curr_label
    else:
        totalimg = np.concatenate((totalimg, imgvec), axis=0)
        totallabel = np.concatenate((totallabel, curr_label), axis=0)
    print("now: %s finished!!:" % imgcnt, end='\r')
    imgcnt += 1
print ("Total %d images loaded." % (imgcnt) , end='\r')

C:\Users\Juno\Desktop\juno_project\Webtoon-Recommender_System./0_Naverimages/main_image_naver/color/


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [15]:
totalimg = totalimg / 255.
height,width = hub.get_expected_image_size(module)
images = tf.reshape(totalimg, shape=[-1,height,width,3])
features = module(tf.cast(images, dtype=tf.float32))

In [17]:
target_image = tf.reshape(images[0], [-1,height,width,3])
target_features = module(tf.cast(target_image, dtype=tf.float32))

In [31]:
# compute the cosine similarity
dot = tf.matmul(features, tf.transpose(target_features))
similarity = tf.divide(dot, tf.reshape(tf.norm(features, axis=1) * tf.norm(target_features),shape=[-1,1]))
similarity = tf.reshape(similarity, [-1])

In [32]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs = sess.run(similarity)

In [34]:
outputs.argsort()[::-1]

array([  0, 309, 335,  54, 631, 528, 661, 100, 543, 203, 208, 106, 676,
       709, 302, 366, 623, 526, 416, 694,   3, 730, 731, 459, 350, 493,
       103, 569, 357, 326, 380,  39,  95, 432, 157, 327, 721,  93, 739,
       454, 225, 443, 170, 502, 355, 635, 271, 699, 134, 401, 688, 315,
        14, 590,  37, 102, 121, 369, 650, 450, 492, 706, 617, 566, 636,
       278, 200, 482, 685, 501, 117, 209,   2, 371, 162, 405, 535, 589,
       116, 507, 740, 144, 578, 560, 490, 541,  47, 268, 250,  27, 689,
       408, 570,  96, 312, 285,  17, 741, 557, 660, 725,  19, 677, 417,
       345, 687, 105, 436, 646, 392, 125, 282, 379,   5, 172, 136, 128,
       343, 729,  10, 260, 620, 274, 386, 475, 736, 194, 423,  44, 231,
       622,  50,  11, 219, 180, 242, 424, 129, 230, 506, 185, 722, 592,
       393,  91, 281, 264, 255, 659, 691, 160,  69, 733, 728, 259, 720,
       473, 139, 126, 484,  94, 715, 323, 228, 239, 648, 619, 421,  90,
       296,  85, 497, 294, 211, 198, 337, 155, 130, 452, 154, 49

In [39]:
flist[0], flist[631]

('103391.png', '702165.png')